# Loading Libraries:

In [1]:
import pygame
from pygame.math import Vector2
from pygame.transform import rotozoom # for scaling and rotating
import random
from pygame.image import load
from pygame.math import Vector2
from pygame.mixer import Sound
from pygame import Color
from pygame.font import Font
import time
import os

pygame 2.0.1 (SDL 2.0.14, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


# Game Object Classes:  

In [2]:
class GameObject:
    def __init__(self, position, sprite, velocity):
        self.position = Vector2(position)
        self.sprite = sprite
        self.radius = sprite.get_width() / 2
        self.velocity = Vector2(velocity)

    def draw(self, surface):
        blit_position = self.position - Vector2(self.radius)
        surface.blit(self.sprite, blit_position)

    def move(self, surface):
        self.position = wrap_position(self.position + self.velocity, surface)

    def collides_with(self, other_obj):
        distance = self.position.distance_to(other_obj.position)
        return distance < self.radius + other_obj.radius

UP = Vector2(0, -1)
DOWN = Vector2(0, 1)

class Spaceship(GameObject):
    MANEUVERABILITY = 5 # defines how fast we can rotate
    ACCELERATION = 0.2 # how the speed changes as i press up key 
    BULLET_SPEED = 3
    def __init__(self, position, create_bullet_callback):
        self.create_bullet_callback = create_bullet_callback
        self.bomb_sound = load_sound("bomb")
        
        self.direction = Vector2(UP)
        super().__init__(position, load_sprite("ship"), Vector2(0))
    
    def rotate(self, clockwise=True):
        sign = 1 if clockwise else -1
        angle = self.MANEUVERABILITY * sign
        self.direction.rotate_ip(angle)
    
    def draw(self, surface):
        angle = self.direction.angle_to(UP)
        rotated_surface = rotozoom(self.sprite, angle, 1.0)
        rotated_surface_size = Vector2(rotated_surface.get_size())
        blit_position = self.position - rotated_surface_size * 0.5
        #print(blit_position)
        surface.blit(rotated_surface, blit_position)
    
    def accelerate(self):
        self.velocity += self.direction * self.ACCELERATION
        #print(self.velocity)
        if self.velocity.x >= 2:
            self.velocity.x = 2
        if self.velocity.y >= 2:
            self.velocity.y = 2
        
    def shoot(self):
        bullet_velocity = self.direction * self.BULLET_SPEED + self.velocity
        bullet = Bullet(self.position, bullet_velocity)
        self.create_bullet_callback(bullet)
        self.bomb_sound.play()
        
class Asteroid(GameObject):
    def __init__(self, position, create_asteroid_callback, size = 3):
        self.bomb_sound = load_sound("asteroid")
        self.create_asteroid_callback = create_asteroid_callback
        self.size = size
        size_to_scale = {
            3: 1,
            2: 0.5,
            1: 0.25,
            }
        scale = size_to_scale[size]
        sprite = rotozoom(load_sprite("asteroid"), 0, scale)
        super().__init__(position, sprite, get_random_velocity(1, 1))
    
    def split(self):
        self.bomb_sound.play()
        if self.size > 1:
            for _ in range(2):
                asteroid = Asteroid(self.position, self.create_asteroid_callback, self.size - 1)
                self.create_asteroid_callback(asteroid)
        
class Bullet(GameObject):
    def __init__(self, position, velocity):
        super().__init__(position, load_sprite("bomb"), velocity)
        
    def move(self, surface):
        self.position = self.position + self.velocity

# Utility Functions :

In [3]:
def load_sprite(name, with_alpha=True):
    path = f"assets/sprites/{name}.png"
    loaded_sprite = load(path)

    if with_alpha:
        return loaded_sprite.convert_alpha()
    else:
        return loaded_sprite.convert()
    
def wrap_position(position, surface): # if we reach the end of screen move it to opposite end
    x, y = position 
    w, h = surface.get_size()
    return Vector2(x % w, y % h)

def get_random_position(surface):
    return Vector2(
        random.randrange(surface.get_width()),
        random.randrange(surface.get_height()),
    )

def get_random_velocity(min_speed, max_speed):
    speed = random.randint(min_speed, max_speed)
    angle = random.randrange(0, 360)
    return Vector2(speed, 0).rotate(angle)

def load_sound(name):
    path = f"assets/sounds/{name}.wav"
    return Sound(path)

def print_text(surface, text, font, color=Color("tomato")):
    text_surface = font.render(text, True, color)

    rect = text_surface.get_rect()
    rect.center = Vector2(surface.get_size()) / 2

    surface.blit(text_surface, rect)

def score(surface, count):
    white = (255,255,255)
    font = Font('freesansbold.ttf', 30)
    text = font.render("Score: "+str(count), True, white)
    surface.blit(text, [10,10])
    
def Hscore(surface, count):
    white = (255,255,255)
    font = Font('freesansbold.ttf', 30)
    text = font.render("High score: "+str(count), True, white)
    surface.blit(text, [530,10])

# GAME: 

In [4]:
class SpaceShooter:
    MIN_ASTEROID_DISTANCE = 250 # distance the asteroid must leave in between them to save the 
                                # ship from destroyed the very instance the game is turned on.
    def __init__(self, level, survival = False, highscore = -1):
        self._init_pygame()
        
        self.flag = True
        self.survival = survival
        self.highscore = highscore
        self.score = 0
        self.level = level
        
        self.font = pygame.font.Font(None, 64)
        self.message = ""
        
        self.screen = pygame.display.set_mode((800, 600)) # width 800 pixels and height 600 
        self.background = load_sprite('space',False)
        self.clock = pygame.time.Clock()
        
        #self.asteroids = [Asteroid((0, 0)) for _ in range(6)]
        self.asteroids = [] #[Asteroid(get_random_position(self.screen)) for _ in range(6)]
        self.bullets = []
        self.Destruct = GameObject((400,300), load_sprite("explosion"), Vector2(0))
        self.spaceship = Spaceship((400, 300), self.bullets.append)
        
        if level == 1: 
            n = 6
        elif level == 2:
            n = 8
        else:
            n = 10
        for _ in range(n):
            while True:
                position = get_random_position(self.screen)
                if (position.distance_to(self.spaceship.position)> self.MIN_ASTEROID_DISTANCE):
                    break
            self.asteroids.append(Asteroid(position, self.asteroids.append))
        
        # self.asteroid = GameObject(
        #     (400, 300), load_sprite("asteroid"), (1, 0)
        # )

    def main_loop(self):
        while self.flag:
            self._handle_input()
            self._process_game_logic()
            self._draw()
            
    def _init_pygame(self):
        pygame.init()
        pygame.display.set_caption("Space Shooter/Survival")

    def _handle_input(self):
        for event in pygame.event.get():
            if event.type == pygame.QUIT or (event.type == pygame.KEYDOWN ## to check input is from keyboard
                                             and event.key == pygame.K_ESCAPE):
                quit()
                pygame.quit()
                
            is_key_pressed = pygame.key.get_pressed()
            if is_key_pressed[pygame.K_r]:
                if self.survival == True:
                    self.__init__(2,True,highscore=self.highscore)
                else:
                    self.__init__(self.level)
            pygame.key.set_repeat(20)
            if self.spaceship:
                if is_key_pressed[pygame.K_RIGHT]:
                    self.spaceship.rotate(clockwise=True)    
                elif is_key_pressed[pygame.K_LEFT]:
                    self.spaceship.rotate(clockwise=False)
                if is_key_pressed[pygame.K_UP]:
                    pygame.key.set_repeat(120)
                    self.spaceship.accelerate()
                elif self.spaceship and event.type == pygame.KEYDOWN and event.key == pygame.K_SPACE \
                    and not self.survival:
                    pygame.key.set_repeat() 
                    self.spaceship.shoot()
                    
              
    def _process_game_logic(self):
        for game_object in self._get_game_objects():
            game_object.move(self.screen)
        #self.spaceship.move(self.screen)
        #self.asteroid.move()
        if self.spaceship:
            for asteroid in self.asteroids:
                if asteroid.collides_with(self.spaceship):
                    self.Destruct.position = self.spaceship.position
                    self.spaceship = None
                    self.message = "You lost!"
                    break
        for bullet in self.bullets[:]:
            for asteroid in self.asteroids[:]:
                if asteroid.collides_with(bullet):
                    self.asteroids.remove(asteroid)
                    self.bullets.remove(bullet)
                    asteroid.split()
                    break
        
        for bullet in self.bullets[:]:
            if not self.screen.get_rect().collidepoint(bullet.position):
                self.bullets.remove(bullet)
        
        if not self.asteroids and self.spaceship:
            self.message = "You won!"

    def _draw(self):
        #self.screen.fill((0, 0, 255))
        self.screen.blit(self.background,(-400,-100))
        
        # self.spaceship.draw(self.screen)
        # #self.asteroid.draw(self.screen)
        for game_object in self._get_game_objects():
            game_object.draw(self.screen)
        
        if self.spaceship:
                self.score += 0.5
                if self.score > self.highscore:
                    self.highscore = self.score
        
        if self.message:
            if self.message == "You lost!":
                self.Destruct.draw(self.screen)
                for game_object in self._get_game_objects():
                    game_object.velocity = Vector2(0)
                    game_object.draw(self.screen)
                print_text(self.screen, self.message, self.font)
            if self.message == 'You won!':
                print_text(self.screen, self.message, self.font)
                pygame.display.update()  
                time.sleep(2)
                if self.level == 3:
                    self.screen.blit(self.background,(-400,-100))
                    print_text(self.screen, 'Thank You Now Try Survival Mode',self.font)
                    pygame.display.update() 
                    time.sleep(5)
                    self.flag = False
                    return 
                    
                for i in range(5):
                    self.screen.blit(self.background,(-400,-100))
                    print_text(self.screen, 'Next Level in {}'.format(5-i)+' seconds',self.font)
                    pygame.display.update()  
                    time.sleep(1)
                    
                self.flag = False
        
        if self.survival:
            score(self.screen ,self.score)
            Hscore(self.screen, self.highscore)
        pygame.display.flip()
        self.clock.tick(80)
        #print("Collides:", self.spaceship.collides_with(self.asteroid))
        
    def _get_game_objects(self):
        #return [*self.asteroids, self.spaceship] adding collision case below:
        game_objects = [*self.asteroids, *self.bullets]
        
        if self.spaceship:
            game_objects.append(self.spaceship)
        return game_objects

# GAME MENU AND PLAY

In [5]:
pygame.init()
pygame.display.set_caption("Space Shooter/Survival")
clock = pygame.time.Clock()
clock.tick(120)
        
def message_to_screen(message, textfont, color):
    my_font = textfont
    my_message = my_font.render(message, 0, color)

    return my_message
    
screen = pygame.display.set_mode((800, 600))
background = load_sprite('space',False)
asteroids = []
asteroids = [Asteroid(get_random_position(screen), None) for _ in range(8)]
    
if __name__ == "__main__":
    menu = True
    selected = "Level Game"
    while menu:
        for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    quit()
                    pygame.quit()
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_UP:
                        selected = "Level Game"
                    elif event.key == pygame.K_DOWN:
                        selected = "Survival Mode"
                    if event.key == pygame.K_RETURN:
                        if selected == "Level Game" or selected == "Survival Mode":
                            menu = False
                    if event.key == pygame.K_ESCAPE:
                        quit()
                        pygame.quit()
        
        screen.blit(background,(-400,-100))
        for game_object in asteroids:
            game_object.draw(screen)
            game_object.move(screen)
            
        title = message_to_screen("SPACE SHOOTER", pygame.font.Font(None, 100), pygame.Color('tomato'))
        controls = message_to_screen("Use The 3 arrow keys to move and SPACE to shoot,", \
                                       pygame.font.Font(None, 30), pygame.Color('tomato'))
        General = message_to_screen("Press R to replay and Esc to escape", \
                                    pygame.font.Font(None, 30), pygame.Color('tomato'))
        
        if selected == "Level Game":
            play = message_to_screen("Level Game", pygame.font.Font('freesansbold.ttf', 64), pygame.Color('white'))
        else:
            play = message_to_screen("Level Game", pygame.font.Font(None, 64), pygame.Color('tomato'))
        if selected == "Survival Mode":
            game2 = message_to_screen("Survival Mode", pygame.font.Font('freesansbold.ttf', 64), pygame.Color('white'))
        else:
            game2 = message_to_screen("Survival Mode", pygame.font.Font(None, 64), pygame.Color('tomato'))
        
        title_rect = title.get_rect()
        controls_1_rect = controls.get_rect()
        controls_2_rect = General.get_rect()
        play_rect = play.get_rect()
        quit_rect = game2.get_rect()
        display_width = 800
    
        screen.blit(title, (display_width/2 - (title_rect[2]/2), 40))
        screen.blit(controls, (display_width/2 - (controls_1_rect[2]/2), 120))
        screen.blit(General, (display_width/2 - (controls_2_rect[2]/2), 140))
        screen.blit(play, (display_width/2 - (play_rect[2]/2), 200))
        screen.blit(game2, (display_width/2 - (quit_rect[2]/2), 260))
        pygame.display.update()
        pygame.display.flip()
        
    x = 1 if selected == "Level Game" else 2
    if x == 1:
        print('LEVEL - 1',end = "")
        space_rocks = SpaceShooter(1)
        space_rocks.main_loop()
        print('Completed!')
        print('LEVEL - 2',end = "")
        space_rocks = SpaceShooter(2)
        space_rocks.main_loop()
        print('Completed!')
        print('LEVEL - 3')
        space_rocks = SpaceShooter(3)    
        space_rocks.main_loop()
        print('Congratualtion You Completed the game ')
    if x == 2:
        space_rocks = SpaceShooter(2, True)
        space_rocks.main_loop()


LEVEL - 1

error: video system not initialized